In [1]:
from tensorflow.keras import optimizers
import tensorflow as tf
import os
# DEFINE TF LOG LEVEL
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# DEFINE GPU RESOUCE FOR TF
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


In [5]:
# CLASS BALANCED AND SCHUFFLED TRAINING SET
import numpy as np
time_train = np.load("./time_train_shuffled_10x.npy")
event_train = np.load("./event_train_shuffled_10x.npy")
image_train = np.load("./image_train_shuffled_10x.npy", mmap_mode="r")


In [19]:
def CreateModel(depth, length, junction, initial_filter, type_of_block, learning_rate):
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "2"
    # THE FOLLOWING CODES LINES MAY NOT WORK ON recent tf version -LINE BEGIN
    from tensorflow.compat.v1.keras.backend import set_session
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    set_session(tf.compat.v1.Session(config=config))
    # -LINE END

    from plexusnet.architecture import PlexusNet

    model = PlexusNet(depth=depth, length=length, junction=junction, n_class=1, final_activation="sigmoid", initial_filter=initial_filter,
                      filter_num_for_first_convlayer=4, input_shape=(512, 512), ApplyLayerNormalization=True, run_all_BN=False, GlobalPooling="avg", type_of_block=type_of_block).model
    model.compile(optimizer=optimizers.Adam(learning_rate),
                  loss=tf.keras.losses.binary_crossentropy, metrics=["accuracy"])
    return model


In [19]:

#
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

# 3 block types x 2 depths options x 2 branching factor option (length) x 3 intial filter options = 36 models with different hyperparameter configurations
model = KerasClassifier(build_fn=CreateModel, verbose=0)
param_grid = dict(
    depth=[5, 6],
    learning_rate=[0.0001],  # defualt
    junction=[3],  # defualt
    batch_size=[16],  # defualt
    length=[2, 3],
    initial_filter=[4, 6, 8],
    type_of_block=["soft_att", "inception", "resnet"],
    epochs=[5])
# 3 fold cross validation using a single CPU core and GPU
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1, cv=3)
# training set has balanced classes after oversampling the BCR class.
grid_result = grid.fit(image_train[:1000], event_train[:1000])


In [ ]:
grid_result.best_params_  # GET THE BEST PARAMETER SET
{'batch_size': 16,
 'depth': 5,
 'epochs': 5,
 'initial_filter': 6,
 'junction': 3,
 'learning_rate': 0.0001,
 'length': 2,
 'type_of_block': 'soft_att'}
